## Chapter 06. Language Model

#### Tokenizer

In [1]:
# 데이터 받기
import requests

res = requests.get('https://github.com/euphoris/datasets/raw/master/imdb.zip')

with open('imdb.zip', 'wb') as f:
    f.write(res.content)

In [2]:
import pandas as pd
df = pd.read_csv('imdb.zip')
df

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
...,...,...
995,I just got bored watching Jessice Lange take h...,0
996,"Unfortunately, any virtue in this film's produ...",0
997,"In a word, it is embarrassing.",0
998,Exceptionally bad!,0


In [3]:
# 토큰화
import tensorflow as tf # 텐서플로의 Tokenizer를 사용한다

# 상위 2000개의 단어까지만 번호로 변환하고 나머지 <unk>로 취급한다
tk = tf.keras.preprocessing.text.Tokenizer(num_words=2000, oov_token='<unk>')

# 단어에 번호를 붙힌다
tk.fit_on_texts(df['review']) # 빈 칸 단위로 바꾼 뒤, 소문자로 바꿔 많이 사용하는 순으로 번호를 붙힘
# 한국어는 빈 칸 단위로 끊으면 안 되기 때문에, 다른 전처리가 필요

# 2000번을 넘어가는 단어는 <unk>로 변환되며, 1번으로 통일
tk.word_index

{'<unk>': 1,
 'the': 2,
 'and': 3,
 'a': 4,
 'of': 5,
 'is': 6,
 'this': 7,
 'i': 8,
 'it': 9,
 'to': 10,
 'in': 11,
 'was': 12,
 'movie': 13,
 'film': 14,
 'that': 15,
 'for': 16,
 'as': 17,
 'but': 18,
 'with': 19,
 'one': 20,
 'on': 21,
 'you': 22,
 'are': 23,
 'not': 24,
 'bad': 25,
 "it's": 26,
 'very': 27,
 'all': 28,
 'just': 29,
 'so': 30,
 'good': 31,
 'at': 32,
 'an': 33,
 'be': 34,
 'there': 35,
 'about': 36,
 'have': 37,
 'by': 38,
 'like': 39,
 'from': 40,
 'if': 41,
 'acting': 42,
 'time': 43,
 'out': 44,
 'his': 45,
 'or': 46,
 'really': 47,
 'great': 48,
 'even': 49,
 'he': 50,
 'who': 51,
 'were': 52,
 'has': 53,
 'see': 54,
 'my': 55,
 'characters': 56,
 'well': 57,
 'most': 58,
 'how': 59,
 'more': 60,
 'no': 61,
 'only': 62,
 'when': 63,
 'ever': 64,
 '10': 65,
 'movies': 66,
 'plot': 67,
 'story': 68,
 'made': 69,
 'some': 70,
 'they': 71,
 'best': 72,
 'because': 73,
 'your': 74,
 'can': 75,
 'also': 76,
 "don't": 77,
 'films': 78,
 'than': 79,
 'its': 80,
 'scrip

In [4]:
tk.word_index['good']

31

In [5]:
tk.index_word[31]

'good'

In [6]:
# 토크나이저를 저장한다
import joblib
joblib.dump(tk, 'tokenizer.pkl')

['tokenizer.pkl']

In [7]:
# 토크나이저 호출
import joblib
tk = joblib.load('tokenizer.pkl')

In [8]:
# 언어 모형에 맞게 데이터 정리

seqs = tk.texts_to_sequences(df['review']) # 텍스트를 번호로 바꿔줌
seqs[0]
# 첫 문장을 단어 번호로 바꿔서 구성

[4, 27, 27, 27, 287, 407, 1217, 13, 36, 4, 1218, 1219, 408, 142]

In [9]:
# 앞에 있는 단어가 들어가면, 이후 나올 단어의 확률을 예측하기 때문에, 연속된 데이터의 형태를 n개의 단위로 정리
data = []
for seq in seqs:
    for i in range(0, len(seq) - 4):
        data.append((seq[i:i+4], seq[i+4]))
# 5-gram 형태로 변환 (4개의 단어를 입력받아, 다음의 단어를 출력)

In [10]:
# 데이터 섞기
import random

random.shuffle(data)
# 유사한 데이터끼리 섞여 있기 때문에, 배치 단위로 학습을 진행할 때 배치 안의 데이터가 모두 비슷하면 학습이 안 되므로 랜덤하게 섞음

In [11]:
data[0]

([251, 24, 168, 66], 44)

In [12]:
# x와 y로 데이터를 나눈다
import numpy as np

xs = np.array([x for x, y in data])
ys = np.array([y for x, y in data])

# 저장
joblib.dump((xs, ys), 'lm-data.pkl')

['lm-data.pkl']

In [13]:
# 준비된 데이터 호출

import joblib
tk = joblib.load('tokenizer.pkl')
xs, ys = joblib.load('lm-data.pkl') # xs; 앞에 나올 단어들, ys; 뒤에 나올 단어

In [14]:
# 언어 모형에 들어갈 임베딩 레이어를 만든다
import tensorflow as tf

# 단어 번호가 1번부터 붙으므로 0번까지 포함하면 총 단어 수에 1을 더해야 한다
NUM_WORD = tk.num_words + 1
# 모델을 만들기 위해, 입력의 크기를 결정해야 하는데, 입력의 크기는 단어의 수에 맞춰진다
# 0번은 텍스트의 길이를 맞춰주기 위해서 0번을 채워 사용하는 토큰

tk.num_words

2000

In [15]:
emb1 = tf.keras.layers.Embedding(
    input_dim=NUM_WORD, # 들어갈 단어의 총 갯수
    output_dim=8, # 만들 단어의 차원 크기, 클수록 성능은 좋지만 과적합이 일어날 수 있음
)
# (예) xs[0] = [2,1119,1,6] 을 one-hot encoding 해야 하지만, 임베딩 레이어를 활용할 시 안해도 됨
# 추후 임베딩만 따로 확인하기 위해 임베딩 레이어를 따로 변수로 지정하여 설정

In [16]:
# 언어 모형을 만든다
lm = tf.keras.Sequential([
    emb1, # 임베딩 레이어
    tf.keras.layers.GlobalAveragePooling1D(),    # 임베딩 레이어의 바로 뒷부분에 추가, 계산의 효율성을 위해 추가, 
                                                 # 한 번에 네 개의 단어가 입력되어 임베딩이 생기고, 임베딩에 에버리지를 적용해줌 (학습 파라미터의 숫자를 줄임)
    tf.keras.layers.Dense(8, activation='relu'), # 은닉층
    tf.keras.layers.Dense(NUM_WORD)              # 출력층 = 입력의 총 갯수와 출력의 크기가 같음 
])

In [17]:
# 모형 확인
lm.summary()
# 첫 번째 None = 모델에 들어가는 데이터의 건수에 관한 부분이므로 무시 가능
# 두 번째 None 은 입력한 단어의 갯수이므로 4, 8 의 형태로 출력
# AveragePooling을 통해 평균을 내 8개로 줄여줌 = 아무 Param이 필요 없음 = 0
# 8 개의 출력을 갖고 dense Layer로 가게 되면 출력의 8 x 8 = 72 로 나옴 (Average Pooling을 안할 시 32 x 8)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 8)           16008     
                                                                 
 global_average_pooling1d (  (None, 8)                 0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 2001)              18009     
                                                                 
Total params: 34089 (133.16 KB)
Trainable params: 34089 (133.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
# 모형을 학습시킨다
lm.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), # loss 지정
    optimizer='adam',
    metrics=['accuracy']
)
# 다항 분류이기 때문에 출력층의 activation을 softmax로 설정해야 하지만, 위에서 생략했기 때문에 loss를 계산할 때, softmax로 activation을 한 것처럼
# 계산해야함. 그렇기 때문에 SparseCategorical Crossentropy로 지정한 뒤, (from_logits=True) 옵션 사용
# activation 이 softmax로 설정되어 있다면 from_logits=True 지정 X
# Tensorflow 의 여러 계산 함수가 자체적으로 softmax를 적용해주는 경우가 많기 때문에 모델에서 softmax를 적용하면 이후 계산에서 번거로운 과정 발생

In [19]:
lm.fit(xs, ys, epochs=1)
# 하나의 epochs만 fit

331/331 [==============================] - 1s 2ms/step - loss: 6.8515 - accuracy: 0.0851


In [20]:
# 모형 저장
lm.save('lm.krs')

INFO:tensorflow:Assets written to: lm.krs\assets


INFO:tensorflow:Assets written to: lm.krs\assets


In [21]:
# 단어 임베딩 확인
e = emb1.embeddings.numpy()
e

array([[-3.8729645e-02,  2.0781245e-02,  4.0416311e-02, ...,
         1.7743055e-02,  3.4751307e-02,  4.5311917e-02],
       [-4.1068035e-01, -3.2668084e-01, -3.4414935e-01, ...,
        -3.2421193e-01,  3.4057826e-01,  3.3284202e-01],
       [-3.2356501e-01, -2.9822266e-01, -3.1237251e-01, ...,
        -2.8984061e-01,  3.2561234e-01,  3.1900981e-01],
       ...,
       [-3.6535315e-02,  1.5987823e-02, -2.6365547e-02, ...,
        -9.5546460e-03, -4.4168383e-03,  7.8537770e-02],
       [-2.2907217e-04, -3.5766780e-02, -6.2176559e-02, ...,
         1.6648181e-02,  7.2397619e-02,  1.5873889e-02],
       [-4.6569109e-04,  4.3570232e-02,  4.1937921e-02, ...,
        -1.3980292e-02, -3.9353907e-02,  2.0474683e-02]], dtype=float32)

In [22]:
# 2001개의 모든 단어들에 대해 8차원으로 임베딩을 만들어 놓은 표
e.shape

(2001, 8)

In [23]:
# 단어 임베딩은 임베딩 레이어의 가중치와 동일하다
import numpy as np

w = emb1.get_weights()[0] # 임베딩 레이어의 가중치만 추출
np.array_equal(e, w)
# 임베딩은 일반적인 레이어의 가중치를 학습하는 것과 동일한 과정으로 진행된다
# 그렇기 때문에, 이 가중치를 추출해 다른 모형에 덮어써도 그 모형의 그 레이어가 이미 학습된 것처럼 만들 수 있다

True

In [24]:
# 임베딩 저장
np.savez('word-emb.npz', emb=e)

#### GlobalAveragePooling1D

In [25]:
import tensorflow as tf
import numpy as np

# GlobalAveragePooling1D는 1번 인덱스를 기준으로 평균을 구한다. 예를 들기 위해 다음과 같은 행렬이 있다고 가정
x = np.array([[[1, 2, 3], [3, 6, 9]]], dtype='float32')
x
# 1과 3의 평균, 2와 6의 평균, 3과 6의 평균을 도출함

array([[[1., 2., 3.],
        [3., 6., 9.]]], dtype=float32)

In [26]:
# 3차원 array
# 1 = 신경망에서 데이터의 건수  = 텍스트의 갯수
# 3 = 하나의 벡터의 길이        = 단어 하나를 몇 개의 숫자로 나타내는가?
# 2 = 이런 벡터가 몇 개 있는가? = 단어가 몇 개냐?
x.shape

(1, 2, 3)

In [27]:
# 이 행렬을 GlobalAveragePooling1D 레이어 통과시키면 다음과 같이 된다

avg = tf.keras.layers.GlobalAveragePooling1D()
avg
# tensorflow의 레이어는 마치 함수처럼 사용할 수 있다

In [28]:
y = avg(x).numpy()
y

array([[2., 4., 6.]], dtype=float32)

In [29]:
y.shape
# 두 단어를 집어넣어서 각각의 임베딩이 나올 때, 임베딩의 평균을 내줌
# 위의 예에서 [1,2,3]이 하나의 단어, [3,6,9]가 하나의 단어

(1, 3)

#### 다음 토큰의 확률 예측

In [30]:
# 준비
import joblib
tk = joblib.load('tokenizer.pkl') # 토크나이저와 단어의 번호
xs, ys = joblib.load('lm-data.pkl') # 텍스트를 신경망 모형에 넣어줄수록 변환한 데이터

# 학습된 모형 호출
import tensorflow as tf
lm = tf.keras.models.load_model('lm.krs')

# 다음에 나올 단어의 확률 예측
x = xs[0:1] # 모델은 여러 데이터를 입력 받을 수 있기 위해 shape을 (4,) 이 아닌 (1,4) 형태로 지정해야 함
y = ys[0]

# x의 4단어를 확인한다
[tk.index_word[i] for i in x[0]]

["there's", 'not', 'enough', 'movies']

In [31]:
# 모형에 넣는다
import numpy as np
logit = lm.predict(x.astype('float32')) # x의 자료형을 실수형으로 변경
logit # (1,2001) = 2001개의 모든 단어에 대한 확률
# 모델을 만들 때 소프트맥스를 해주지 않아서 확률이 아님

1/1 [==============================] - 0s 78ms/step


array([[-3.1920993,  3.1606002,  2.7681746, ..., -3.2598271, -3.2116275,
        -3.2491329]], dtype=float32)

In [32]:
# 소프트맥스 함수를 적용하여 확률로 바꾼다
p = tf.nn.softmax(logit).numpy()
p
# p[0, 57] = 57번째 단어의 확률
# 학습된 데이터를 토대로 했을 때, x 뒤에 57번째 단어가 나올 확률

array([[7.4038006e-05, 4.2500786e-02, 2.8705738e-02, ..., 6.9189598e-05,
        7.2606192e-05, 6.9933521e-05]], dtype=float32)

In [33]:
# 여기에서 실제로 나온 단어를 확인한다
tk.index_word[y]

'out'

In [34]:
# 해당 단어의 확률을 본다
p[0, y]

0.0027336762

In [35]:
# 확률이 가장 높은 단어를 알아본다
i = p.argmax()
i

1

In [36]:
p[0, i]

0.042500786

In [37]:
tk.index_word[i]

'<unk>'

### Transfer Learning (전이 학습)

In [38]:
# IMDB 리뷰 데이터
import pandas as pd
df = pd.read_csv('https://github.com/euphoris/datasets/raw/master/imdb.zip')
# 이전의 단어 문서 행렬을 활용한 감성 분석이 아닌, 언어 모형을 활용한 감성 분석 진행

# 토크나이저 호출
import joblib
tk = joblib.load('tokenizer.pkl')

# 텍스트를 토큰의 번호 시퀀스로 변환
seqs = tk.texts_to_sequences(df['review'])
seqs[0]

[4, 27, 27, 27, 287, 407, 1217, 13, 36, 4, 1218, 1219, 408, 142]

In [39]:
# 시퀀스마다 길이가 모두 다르므로 앞에 0을 채워(padding) 길이를 맞춘다
import tensorflow as tf
pads = tf.keras.preprocessing.sequence.pad_sequences(seqs)
# 가장 긴 텍스트를 기준으로 길이가 똑같아 지도록 앞에 0을 채워넣음

In [40]:
# 단어 임베딩 호출
import numpy as np

z = np.load('word-emb.npz')
e = z['emb'] # 단어 임베딩

In [41]:
# 감성 분석 (언어 모형에서 학습되었던 학습을 전이시킴)
emb2 = tf.keras.layers.Embedding(
    input_dim=tk.num_words + 1, # 단어의 갯수
    output_dim=8,               # 8차원으로 출력
    embeddings_initializer=tf.keras.initializers.Constant(e) # 언어 모형 학습 시 사용한 가중치로 세팅
)
# 감성 분석 모형에 들어갈 임베딩 레이어를 만든다. 언어 모형에서 학습된 가중치로 초기화한다
# 다음 단어 예측 기준 학습이 아닌, 감성 예측 기준으로 학습

In [42]:
# 감성 분석 모형 만들기
model = tf.keras.Sequential([
    emb2,
    tf.keras.layers.GlobalAveragePooling1D(), # 단어마다 임베딩 값이 주어질 시, 단어가 굉장히 많기 때문에, 텍스트 안의 단어 임베딩을 평균
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') # 출력 Layer를 이진분류이기 때문에 activation 을 sigmoid로 지정
])

In [43]:
# 모형 요약 확인
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 8)           16008     
                                                                 
 global_average_pooling1d_2  (None, 8)                 0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_2 (Dense)             (None, 8)                 72        
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 16089 (62.85 KB)
Trainable params: 16089 (62.85 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [44]:
# 모형 설정
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [45]:
# 모형 학습
y = df['sentiment'].values # sentiment열의 값을 array 형태로 지정

In [46]:
model.fit(pads, y)

32/32 [==============================] - 0s 1ms/step - loss: 0.6930 - accuracy: 0.5020


### Word Embedding

#### FastText

In [47]:
# 데이터 다운로드
import pandas as pd
nsmc = pd.read_csv('https://github.com/e9t/nsmc/raw/master/ratings_train.txt ', sep='\t')

# 전처리
import re # 정규 표현식 활용

# 한글만 찾아서 추출
def find_hangul(text):
    return re.findall(r'[ㄱ-ㅎ가-힣]+', text) # 정규 표현식을 활용한 적용되는 모든 것을 찾음
# ㄱ-ㅎ, 가-힣까지 글자 중 하나라도 있으면 찾음

data = nsmc[nsmc['document'].notnull()]['document'].map(find_hangul)
# notnull 을 통해 비어있는 함수의 경우 False, 안 비어있으면 True
# .map을 통해 함수 적용 (document의 모든 항목에 함수 적용)

data[0]
# FastText의 경우, 단어 안의 글자 단위로 임베딩을 진행하므로, 굳이 형태소 분석을 진행하지 않아도 비슷한 효과를 냄

['아', '더빙', '진짜', '짜증나네요', '목소리']

In [48]:
# 한글이 아닌 글자를 지우고 공백을 하나로 합침
def only_hangul(text):
    return ' '.join(find_hangul(text)) #리스트의 사이 사이에 빈칸을 하나씩 넣은 뒤 합침

data2 = nsmc[nsmc['document'].notnull()]['document'].map(only_hangul)
data2[0]

'아 더빙 진짜 짜증나네요 목소리'

In [49]:
with open('nsmc.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(data2)) # 여러 행을, 행 사이사이마다 나눠서 저장
# FastText는 전처리한 것을 파일로 불러와도 적용되지만, 공백을 끼워넣어 줘야됨 = only_hangul

In [53]:
# FastText 모형 학습
from gensim.models.fasttext import FastText
from gensim.models.word2vec import Word2Vec

# FastText 모형 생성
model = FastText(vector_size=16)
# size : 임베딩의 크기 (기본값 100) ,단어 하나마다 n차원으로 학습, 학습량: 단어 갯수 * n 차원
# sg : 0 이면 CBOW (기본값), 1이면 Skip-gram
# alpha: 학습률 (기본값 0.025)
# min_alpha: 최소 학습률. FastText는 학습과정에서 학습률을 이 수준까지 점점 낮춘다 (기본값 0.0001)
# window: 문장 내 주변 단어와 대상 단어의 최대 거리 (기본값 5)
# min_count: 임베딩을 학습할 단어의 최소 출현 빈도 (기본값 5)
# Word2Vec도 사용방법은 같다

In [60]:
# 어휘 파악, 파일로 저장한 경우 senetece=data 대신 corpus_file='nsmc.txt'
model.build_vocab(corpus_file='nsmc.txt')
# 어떤 단어로 학습하는가? 어휘 목록 학습

In [62]:
# 모형 학습
model.train(
    corpus_file='nsmc.txt', # 데이터 입력
    epochs=5,       # 에포크 지정
    total_examples=model.corpus_count,
    total_words=model.corpus_total_words
)

(4011324, 5847440)

In [63]:
# 저장
model.save('nsmc.fasttext')

In [64]:
# 불러오기
model = FastText.load('nsmc.fasttext')

#### FastText Embedding

In [66]:
# 모형 불러오기
from gensim.models.fasttext import FastText

model = FastText.load('nsmc.fasttext')

# 단어 임베딩
'히어로' in model.wv.key_to_index
# '히어로'는 단어 임베딩이 학습되어 있다
# model.wv.vocab = 각 어절들의 임베딩이 dictionary 형태로 有

True

In [67]:
model.wv['히어로']
# 히어로를 나타내는 16개의 숫자

array([-0.3049521 ,  0.5861554 ,  0.26280412,  0.52573955,  0.9322114 ,
       -0.07087298, -0.9840946 , -0.36231336,  0.147674  ,  0.16906758,
        0.0853401 , -0.9965359 ,  0.3709958 , -0.34485692,  0.0499838 ,
        0.5252803 ], dtype=float32)

In [68]:
'슈퍼히어로' in model.wv.key_to_index
# '슈퍼히어로'는 단어 임베딩이 없지만

False

In [69]:
# 준단어 토큰의 임베딩을 더해서 임베딩을 계산해준다
# 학습이 안 되이었어도 글자단위 n-gram으로 쪼개, n-gram의 임베딩을 더해 계산
model.wv['슈퍼히어로']

array([-0.16042814,  0.24366389,  0.17598806,  0.23353092,  0.33474842,
        0.01218682, -0.3438654 , -0.16338958,  0.11673915,  0.03473993,
        0.07153479, -0.35117054,  0.11864723, -0.11423315,  0.05896696,
        0.250619  ], dtype=float32)

In [70]:
# 유사도
model.wv.similarity('슈퍼히어로', '히어로')
# '히어로'와 '슈퍼히어로'의 유사도는 높다

0.9813293

In [71]:
from sklearn.metrics.pairwise import cosine_similarity # 코사인 유사도 평가 패키지
# 안 쓰고 모델 내장 유사도 계산 기능을 사용해도 됨

model.wv.similarity('히어로', '평론가')
# '히어로'와 '평론가'의 유사도는 상대적으로 낮다

0.5460062

In [72]:
model.wv.most_similar('평론가')
# '평론가'와 비슷한 단어들

[('점대지', 0.9895889163017273),
 ('점이야', 0.9863005876541138),
 ('평론가들', 0.9855533838272095),
 ('점대야', 0.9833608865737915),
 ('점이냐', 0.9829610586166382),
 ('점대나', 0.9829360246658325),
 ('평론', 0.9824333786964417),
 ('점대면', 0.9812780022621155),
 ('점이나', 0.9801117777824402),
 ('점대는', 0.9773420691490173)]

#### Sentiment Analysis with FastText

In [73]:
# 학습된 FastText 모형 호출
from gensim.models.fasttext import FastText
ft = FastText.load('nsmc.fasttext')
nsmc = pd.read_csv('https://github.com/e9t/nsmc/raw/master/ratings_train.txt ', sep='\t')

In [74]:
# 전처리
df = nsmc[nsmc['document'].notnull()] # 리뷰가 있는 데이터만 선택

In [75]:
from sklearn.model_selection import train_test_split
doc_train, doc_test, y_train, y_test = train_test_split(df['document'], df['label'], test_size=0.2, random_state=42)
# 훈련용 데이터와 테스트용 데이터 분할
# 원래는 임베딩 학습 전에, 데이터 분할을 먼저 진행했어야 함 (트레인 데이터를 활용해 임베딩)

In [76]:
# 한글만 추출하는 함수
import re
def find_hangul(text):
    return re.findall(r'[ㄱ-ㅎ가-힣]+', text)

In [77]:
import numpy as np
x_train = np.zeros((1000, 16))
# 1000, 16 크기의 행렬을 만듦 (튜플 형태) (doc_train의 shape은 119996, 인데 1000개의 데이터만 뽑아서 16차원으로 표현)
# 단어 문서 행렬은 단어의 수를 세, 행렬을 만들지만, 임베딩을 이용해 행렬 형태로 변환함
# 단어마다 임베딩 값을 평균내 문서의 임베딩으로 사용 = GlobalAverage 의 수동 형태

In [78]:
for i, doc in enumerate(doc_train.iloc[:1000]): # doc_train에서 1,000개의 행만 사용
    vs = [ft.wv[word] for word in find_hangul(doc) if word in ft.wv] # doc_trian에서 뽑힌 행이 단어 하나하나가 되서(word), 학습된 Fasttext의 단어 임베딩에서 word에 해당되는 단어를 찾고, 없으면 만듦 
    if vs:
        x_train[i,] = np.mean(vs, axis=0) # 16개의 차원을, 각각의 차원에 대해 평균을 내, x_train의 i번째 행에 덮어씀
# 각 문서에서 한글 단어를 찾아 단어 임베딩을 구하고, 이를 문서마다 평균을 낸다

In [79]:
x_train[0]
# 각 문서의 단어 임베딩 평균

array([-0.82882804,  0.31164083,  0.21903975,  1.14677274,  1.19770491,
        0.71183288, -1.05152655, -1.34460402,  1.07707155, -0.52714121,
        0.34436479, -0.54720414, -1.77543116,  0.02745596,  0.92354923,
        0.73833674])

In [80]:
# 각 문서의 단어 임베딩 평균을 이용하여 감성을 예측하는 모형을 만든다
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])
# Embedding과 GlobalAveragePooling1D가 없지만, 데이터에서 직접 수행함
# FastText의 경우, 토큰이 없으면 변환하는 과정이 있기 때문에, 그냥 임베딩 레이어에 덮어쓰는 형태로는 진행하기 힘듦

In [81]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [82]:
model.fit(x_train, y_train.values[:1000], epochs=1) # x_train의 갯수에 맞게 y_train 갯수 설정
# 모델이 단순해진다는 장점 (모델의 파라미터 수가 적음)
# 원래는 단어가 들어오면 임베딩이 들어오고 등등... FastText를 활용하면, 몇 천개의 단어를 활용하지 않고 임베딩만을 사용하기 때문에
# 신경망 모델이 단순해져, 효율성을 높일 수 있음

32/32 [==============================] - 0s 1ms/step - loss: 0.8001 - accuracy: 0.4520


#### Sentiment Analysis with RNN

In [84]:
# 데이터 준비
import pandas as pd
df = pd.read_csv('./data/imdb.zip')

import joblib
tk = joblib.load('tokenizer.pkl')

# 데이터 분할
from sklearn.model_selection import train_test_split
review_train, review_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=2023)

In [85]:
# 토큰화
seqs = tk.texts_to_sequences(review_train) # 학습용 리뷰 데이터 토큰화
seqs[0] #review_train.iloc[0] 이 번호로 변환

[2,
 982,
 32,
 1,
 52,
 956,
 3,
 68,
 6,
 30,
 137,
 9,
 1,
 987,
 726,
 36,
 94,
 1190,
 1,
 4,
 1,
 1,
 2,
 1,
 10,
 185,
 663,
 675,
 8,
 29,
 89,
 385,
 36,
 97,
 5,
 7,
 14,
 3,
 369,
 24,
 10,
 396,
 1,
 213,
 4,
 1,
 518,
 10,
 1,
 1191,
 1]

In [86]:
# 순방향 순환신경망
import tensorflow as tf
pads = tf.keras.preprocessing.sequence.pad_sequences(seqs, maxlen=None, padding='pre', truncating='pre')
# 순환신경망에 넣기 전에, 신경망을 한 번에 처리하기 위해 길이를 일정하게 맞춰주는 패딩 작업 진행
# 패딩 진행, 길이가 짧으면 앞쪽에 0을 채운다(padding='pre')
# maxlen은 최대 길이를 지정할 수 있다. 지정하지 않으면 가장 긴 문자열의 길이로 지정된다
# truncating='pre'는 maxlen보다 긴 문자열일 경우 앞쪽을 자른다. 뒤쪽을 자르게 하려면 'post'로 설정

In [87]:
NUM_WORDS= tk.num_words + 1

In [88]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(NUM_WORDS, 8, mask_zero=True), # 임베딩 레이어를 우선 배치, 단어의 배치(NUM_WORDS), 몇 차원으로 임베딩 할것인가?(8)
    tf.keras.layers.LSTM(8), # 순환신경망의 레이어 설정
    tf.keras.layers.Dense(1, activation='sigmoid') # 마지막 출력 레이어
])
# Embedding에서 mask_zero=True로 설정하면 0으로 패딩된 부분의 예측은 손실에 반영하지 않는다

In [89]:
model.summary() # 데이터가 한 번에 하나씩 들어갈 수도, 열 개씩 들어갈 수도 있으므로 첫 번째 None은 미지수이다
                # pads.shape = (800, 73)이므로 800건에 대해 73개의 토큰을 구성하고 있으므로, 두 번째 None은 73이 된다
                # 73개의 토큰을 받아, 각각 8차원의 임베딩으로 변환
                # LSTM으로 들어가, 73개의 토큰을 순차적으로 처리한 뒤 8개의 출력으로 만듦
                # 73개 토큰을 입력받아 마지막으로 출력하는데, 패딩을 뒤에 하면, 문장이 끝난 뒤 지속적으로 0이 입력되므로, LSTM 레이어에서 정보가 손실되어 0에 영향을 받을 수밖에 없음
                # = 0을 앞으로 붙히는 이유

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 8)           16008     
                                                                 
 lstm (LSTM)                 (None, 8)                 544       
                                                                 
 dense_6 (Dense)             (None, 1)                 9         
                                                                 
Total params: 16561 (64.69 KB)
Trainable params: 16561 (64.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [90]:
# 모델 학습
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [91]:
model.fit(pads, y_train.values, epochs=10)

Epoch 1/10
25/25 [==============================] - 2s 14ms/step - loss: 0.6921 - accuracy: 0.5525
Epoch 2/10
25/25 [==============================] - 0s 14ms/step - loss: 0.6865 - accuracy: 0.7225
Epoch 3/10
25/25 [==============================] - 0s 14ms/step - loss: 0.6742 - accuracy: 0.7950
Epoch 4/10
25/25 [==============================] - 0s 14ms/step - loss: 0.6349 - accuracy: 0.7900
Epoch 5/10
25/25 [==============================] - 0s 14ms/step - loss: 0.5314 - accuracy: 0.8575
Epoch 6/10
25/25 [==============================] - 0s 13ms/step - loss: 0.4302 - accuracy: 0.8863
Epoch 7/10
25/25 [==============================] - 0s 13ms/step - loss: 0.3605 - accuracy: 0.9187
Epoch 8/10
25/25 [==============================] - 0s 14ms/step - loss: 0.3012 - accuracy: 0.9413
Epoch 9/10
25/25 [==============================] - 0s 14ms/step - loss: 0.2583 - accuracy: 0.9525
Epoch 10/10
25/25 [==============================] - 0s 15ms/step - loss: 0.2182 - accuracy: 0.9688


In [92]:
# 역방향 순환신경망
pads = tf.keras.preprocessing.sequence.pad_sequences(seqs, padding='post')
# 패딩 진행, 길이가 짧으면 뒤쪽에 0을 채운다 (padding='post')

In [93]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(NUM_WORDS, 8, mask_zero=True),
    tf.keras.layers.LSTM(8, go_backwards=True),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [94]:
# 양방향 순환신경망
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(NUM_WORDS, 8, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(8)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
# LSTM을 Bidirectinal로 감싸주면 자동으로 순방향과 역방향 레이어를 넣어준다

In [95]:
model.summary()
# 중간 레이어가 두 개로 복사되기 때문에, 순방향의 8차원 출력과 역방향의 8차원 출력이 나오기 때문에 16개의 출력이 나오게 됨

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 8)           16008     
                                                                 
 bidirectional (Bidirection  (None, 16)                1088      
 al)                                                             
                                                                 
 dense_8 (Dense)             (None, 1)                 17        
                                                                 
Total params: 17113 (66.85 KB)
Trainable params: 17113 (66.85 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
